<a href="https://colab.research.google.com/github/Samgomes2510/dashborad-ecommerce/blob/main/2ConsultasAvancadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [4]:
from google.colab import files
uploaded = files.upload()

Saving avaliacoes_pedido.csv to avaliacoes_pedido (1).csv
Saving clientes.csv to clientes (1).csv
Saving itens_pedido.csv to itens_pedido (1).csv
Saving pagamentos_pedido.csv to pagamentos_pedido (1).csv
Saving pedidos.csv to pedidos (1).csv
Saving produtos.csv to produtos (1).csv
Saving vendedores.csv to vendedores (1).csv


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
spark = SparkSession.builder.getOrCreate()

In [7]:
produtos = spark.read.csv("produtos.csv", header=True, inferSchema=True)
vendedores = spark.read.csv("vendedores.csv", header=True, inferSchema=True)
clientes = spark.read.csv("clientes.csv", header=True, inferSchema=True)
itens_pedido = spark.read.csv("itens_pedido.csv", header=True, inferSchema=True)
pagamentos_pedido = spark.read.csv("pagamentos_pedido.csv", header=True, inferSchema=True)
avaliacoes_pedido = spark.read.csv("avaliacoes_pedido.csv", header=True, inferSchema=True)
pedidos = spark.read.csv("pedidos.csv", header=True, inferSchema=True)

In [8]:
itens_pedido.show()
pagamentos_pedido.show()

+--------------------+--------------+--------------------+--------------------+-------------------+------+-----------+
|           id_pedido|item_id_pedido|          id_produto|         id_vendedor|  data_limite_envio| preco|valor_frete|
+--------------------+--------------+--------------------+--------------------+-------------------+------+-----------+
|00010242fe8c5a6d1...|             1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|      13.29|
|00018f77f2f0320c5...|             1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|      19.93|
|000229ec398224ef6...|             1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|      17.87|
|00024acbcdf0a6daa...|             1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|      12.79|
|00042b26cf59d7ce6...|             1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|      18.14|
|00048cc3ae777c65d...|             1|ef92defde84

In [11]:
pagamentos_pedido.createOrReplaceTempView('pagamentos_pedido')
salario_medio_sql = spark.sql("""
SELECT*
FROM pagamentos_pedido
WHERE valor_pagamento >(
  SELECT AVG(valor_pagamento)
  FROM pagamentos_pedido
)
""")

salario_medio_df = pagamentos_pedido.agg(avg('valor_pagamento').alias('pagamento_medio'))

salario_medio = pagamentos_pedido.agg(avg('valor_pagamento').alias('pagamento_medio')).collect()[0][0] #Carregar a lista para o drive
pedidos_filtrados_df = pagamentos_pedido.filter(pagamentos_pedido['valor_pagamento'] > salario_medio)

#Corrected variable names to match where they were defined
salario_medio_df.show() #Changed avg_salary_df to salario_medio_df
print(salario_medio) #Changed avg_salary to salario_medio
pedidos_filtrados_df.show() #Changed df_filtered to pedidos_filtrados_df


+------------------+
|   pagamento_medio|
+------------------+
|154.10038041698573|
+------------------+

154.10038041698573
+--------------------+-------------------+--------------+------------------+---------------+
|           id_pedido|sequencia_pagamento|tipo_pagamento|parcelas_pagamento|valor_pagamento|
+--------------------+-------------------+--------------+------------------+---------------+
|1f78449c87a54faf9...|                  1|   credit_card|                 6|         341.09|
|d88e0d5fa41661ce0...|                  1|   credit_card|                 8|         188.73|
|12e5cfe0e4716b59a...|                  1|   credit_card|                10|         157.45|
|8ac09207f415d55ac...|                  1|   credit_card|                 4|         244.15|
|4214cda550ece8ee6...|                  1|   credit_card|                 2|         170.57|
|4d680edbaa7d3d9be...|                  1|   credit_card|                10|         353.09|
|8cd68144cdb62dc0d...|                

In [12]:
maior_preco_por_vendedor_df = itens_pedido.groupBy("id_vendedor").agg(max("preco").alias("max_preco"))

pedidos_com_maior_preco = itens_pedido.join(maior_preco_por_vendedor_df,
                                             (itens_pedido["id_vendedor"] == maior_preco_por_vendedor_df["id_vendedor"]) &
                                             (itens_pedido["preco"] == maior_preco_por_vendedor_df["max_preco"])
).select(itens_pedido["*"])

maior_preco_por_vendedor_df.show()
pedidos_com_maior_preco.show()

+--------------------+---------+
|         id_vendedor|max_preco|
+--------------------+---------+
|ff063b022a9a0aab9...|   1230.0|
|8e6cc767478edae94...|    371.2|
|a49928bcdf77c55c6...|     89.9|
|da7039f29f90ce5b4...|    128.0|
|062ce95fa2ad4dfae...|    369.9|
|2009a095de2a2a416...|     36.9|
|0ea22c1cfbdc755f8...|     99.9|
|6eeed17989b0ae47c...|    279.0|
|e63e8bfa530fb1691...|     32.9|
|4d600e08ecbe08258...|   699.17|
|9803a40e82e45418a...|  2499.75|
|b3f19518fcec265b2...|    99.99|
|ec8879960bd2221d5...|    254.9|
|0b64bcdb0784abc13...|     35.9|
|c522be04e020c1e7b...|   139.99|
|9c068d10aca38e85c...|    540.0|
|297d5eccd19fa9a83...|   114.99|
|9b1050e85becf3ae9...|    42.57|
|a3082f442524a1be4...|     95.9|
|e38db885400cd35c7...|     64.9|
+--------------------+---------+
only showing top 20 rows

+--------------------+--------------+--------------------+--------------------+-------------------+------+-----------+
|           id_pedido|item_id_pedido|          id_produto|     

In [13]:
pedido_id_inicial = itens_pedido.withColumn("id_incial", regexp_extract("id_pedido", r"^(.{5})", 1))
pedido_id_inicial.select("id_pedido", "id_incial").show()

+--------------------+---------+
|           id_pedido|id_incial|
+--------------------+---------+
|00010242fe8c5a6d1...|    00010|
|00018f77f2f0320c5...|    00018|
|000229ec398224ef6...|    00022|
|00024acbcdf0a6daa...|    00024|
|00042b26cf59d7ce6...|    00042|
|00048cc3ae777c65d...|    00048|
|00054e8431b9d7675...|    00054|
|000576fe39319847c...|    00057|
|0005a1a1728c9d785...|    0005a|
|0005f50442cb953dc...|    0005f|
|00061f2a7bc09da83...|    00061|
|00063b381e2406b52...|    00063|
|0006ec9db01a64e59...|    0006e|
|0008288aa423d2a3f...|    00082|
|0008288aa423d2a3f...|    00082|
|0009792311464db53...|    00097|
|0009c9a17f916a706...|    0009c|
|000aed2e25dbad2f9...|    000ae|
|000c3e6612759851c...|    000c3|
|000e562887b1f2006...|    000e5|
+--------------------+---------+
only showing top 20 rows



In [14]:
from os import truncate
vendedores_anonimo_df = itens_pedido.withColumn("id_vendedor_anonymized", regexp_replace("id_vendedor", "[a-zA-Z]", "X"))
vendedores_anonimo_df.select("id_vendedor", "id_vendedor_anonymized").show(truncate=False)

+--------------------------------+--------------------------------+
|id_vendedor                     |id_vendedor_anonymized          |
+--------------------------------+--------------------------------+
|48436dade18ac8b2bce089ec2a041202|48436XXXX18XX8X2XXX089XX2X041202|
|dd7ddc04e1b6c2c614352b383efe2d36|XX7XXX04X1X6X2X614352X383XXX2X36|
|5b51032eddd242adc84c38acab88f23d|5X51032XXXX242XXX84X38XXXX88X23X|
|9d7a1d34a5052409006425275ba1c2b4|9X7X1X34X5052409006425275XX1X2X4|
|df560393f3a51e74553ab94004ba5c87|XX560393X3X51X74553XX94004XX5X87|
|6426d21aca402a131fc0a5d0960a3c90|6426X21XXX402X131XX0X5X0960X3X90|
|7040e82f899a04d1b434b795a43b4617|7040X82X899X04X1X434X795X43X4617|
|5996cddab893a4652a15592fb58ab8db|5996XXXXX893X4652X15592XX58XX8XX|
|a416b6a846a11724393025641d4edd5e|X416X6X846X11724393025641X4XXX5X|
|ba143b05f0110f0dc71ad71b4466ce92|XX143X05X0110X0XX71XX71X4466XX92|
|cc419e0650a3c5ba77189a1882b7556a|XX419X0650X3X5XX77189X1882X7556X|
|8602a61d680a10a82cceeeda0d99ea3d|8602X61X680X10

In [16]:
def calcular_total(preco, frete):
  return preco + frete

  calcular_soma_udf = udf(calcular_total, FloatType())

  pedidos_valor_total_df = itens_pedido.withColumn("valor_total", round(calcular_soma_udf(itens_pedido["preco"], itens_pedido["frete"]),2))
  pedidos_valor_total_df.show()

  df_with_total = itens_pedido.withColumn("valor_total", round(col("preco"), col("valor_frete"),2))

In [18]:
def classificar_preco(preco):
  if preco < 50:
    return "Baixo"
  elif preco < 500:
    return "Médio"
  else:
    return "Alto"

classificar_preco_udf = udf(classificar_preco, StringType())

pedidos_categoria_df = itens_pedido.withColumn("categoria_preco", classificar_preco_udf(itens_pedido["preco"]))
pedidos_categoria_df.show()

pedido_categoria_df = itens_pedido.withColumn(
    "categoria_preco",
     when(col("preco") < 50, "Baixo")
     .when((col("preco") >= 50) & (col("preco") < 500), "Médio")
     .otherwise("Alto")
)

+--------------------+--------------+--------------------+--------------------+-------------------+------+-----------+---------------+
|           id_pedido|item_id_pedido|          id_produto|         id_vendedor|  data_limite_envio| preco|valor_frete|categoria_preco|
+--------------------+--------------+--------------------+--------------------+-------------------+------+-----------+---------------+
|00010242fe8c5a6d1...|             1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|      13.29|          Médio|
|00018f77f2f0320c5...|             1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|      19.93|          Médio|
|000229ec398224ef6...|             1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|      17.87|          Médio|
|00024acbcdf0a6daa...|             1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|      12.79|          Baixo|
|00042b26cf59d7ce6...|             1|ac6c3623068f30de0.

In [19]:
spark.stop()